In [ ]:
!pip install requests bs4 pandas numpy datetime urllib3 tqdm reachability rpy2


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.5/52.5 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.3/247.3 kB 10.5 MB/s eta 0:00:00


In [ ]:
import requests
from bs4 import BeautifulSoup as beautifulsoup
import json
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from urllib.parse import urlencode
from tqdm import tqdm
import sys
import time
from reachability import reachability
import pickle

url = "https://www.courtlistener.com/api/rest/v3/"
api_key = "163cea228fb27936988d579ed72fe787848e1866"

def get_data_from_clusters(api_key, url):
    url = url + "clusters/"
    headers = {"Authorization": f"Token {api_key}"}
    params = {
        "dockets__court": "scotus",
        "id__gt": 0,  # Start from the first ID
        "date_filed__gt": "2019-01-01",
        "sub_opinions__isnull": False,
        "sub_opinions__author__isnull": False,
        "order_by": "id",
        "fields": "id,case_name,date_filed,scdb_decision_direction,scdb_votes_majority,scdb_votes_minority,sub_opinions"
    }

    clusters_data = []
    with tqdm(desc="Fetching clusters data", unit="page") as pbar:
        while True:
            try:
                response = requests.get(
                    url, headers=headers, params=urlencode(params, safe=",")
                )
                response.raise_for_status()
                data = response.json()
                for cluster in data["results"]:
                    sub_opinions = cluster["sub_opinions"]
                    cluster["sub_opinions"] = [
                        int(sub_opinion.split("/")[-2]) for sub_opinion in sub_opinions
                    ]
                clusters_data.extend(data["results"])
                if not data["next"]:
                    break
                params["id__gt"] = data["results"][-1]["id"]
                time.sleep(1)  # Add a delay to avoid being banned
                pbar.update(1)
            except requests.exceptions.HTTPError as err:
                print(err)
                time.sleep(180)
                continue
            except requests.exceptions.ConnectionError as err:
                reachable = reachability(timeout=None)
                while not reachable.is_online():
                    print("Waiting for internet connection...")
                    time.sleep(120)
                continue
            except Exception as err:
                print("Other error: ", err)
                break
    return clusters_data

# Get data from clusters
clusters_data = get_data_from_clusters(api_key, url)

# Pickle the clusters_data
with open('clusters_data.pickle', 'wb') as file:
    pickle.dump(clusters_data, file)

print("Clusters data has been pickled and saved to 'clusters_data.pickle'.")

Fetching clusters data: 121page [09:29,  4.33s/page]

502 Server Error: Bad Gateway for url: https://www.courtlistener.com/api/rest/v3/clusters/?dockets__court=scotus&id__gt=4618637&date_filed__gt=2019-01-01&sub_opinions__isnull=False&sub_opinions__author__isnull=False&order_by=id&fields=id,case_name,date_filed,scdb_decision_direction,scdb_votes_majority,scdb_votes_minority,sub_opinions


Fetching clusters data: 889page [1:05:19,  3.72s/page]

504 Server Error: Gateway Time-out for url: https://www.courtlistener.com/api/rest/v3/clusters/?dockets__court=scotus&id__gt=5287107&date_filed__gt=2019-01-01&sub_opinions__isnull=False&sub_opinions__author__isnull=False&order_by=id&fields=id,case_name,date_filed,scdb_decision_direction,scdb_votes_majority,scdb_votes_minority,sub_opinions


Fetching clusters data: 963page [1:13:06,  3.75s/page]

...

502 Server Error: Bad Gateway for url: https://www.courtlistener.com/api/rest/v3/clusters/?dockets__court=scotus&id__gt=9381892&date_filed__gt=2019-01-01&sub_opinions__isnull=False&sub_opinions__author__isnull=False&order_by=id&fields=id,case_name,date_filed,scdb_decision_direction,scdb_votes_majority,scdb_votes_minority,sub_opinions


Fetching clusters data: 1327page [1:38:55,  1.84s/page]

504 Server Error: Gateway Time-out for url: https://www.courtlistener.com/api/rest/v3/clusters/?dockets__court=scotus&id__gt=9388714&date_filed__gt=2019-01-01&sub_opinions__isnull=False&sub_opinions__author__isnull=False&order_by=id&fields=id,case_name,date_filed,scdb_decision_direction,scdb_votes_majority,scdb_votes_minority,sub_opinions


Fetching clusters data: 1544page [1:48:20,  4.21s/page]

Clusters data has been pickled and saved to 'clusters_data.pickle'.
